# Scratch space for Pull Request (PR) to PennyLane

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np

import pennylane as qml
from pennylane import expval

In [2]:
qml.about()

Name: PennyLane
Version: 0.9.0.dev0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/XanaduAI/pennylane
Author: None
Author-email: None
License: Apache License 2.0
Location: /Users/hannahsim/repos/pennylane
Requires: numpy, scipy, networkx, autograd, toml, appdirs, semantic-version
Required-by: PennyLane-Qchem
Platform info:           Darwin-18.6.0-x86_64-i386-64bit
Python version:          3.7.5
Numpy version:           1.17.2
Scipy version:           1.2.1
Installed devices:
- default.gaussian (PennyLane-0.9.0.dev0)
- default.qubit (PennyLane-0.9.0.dev0)
- default.tensor (PennyLane-0.9.0.dev0)
- default.tensor.tf (PennyLane-0.9.0.dev0)


In [3]:
dev = qml.device("default.qubit", wires=2)

In [23]:
alpha = 0.4
beta = 0.2

coeffs = [alpha, alpha, beta]
obs_list = [
    qml.PauliZ(0) @ qml.Identity(1),
    qml.Identity(0) @ qml.PauliZ(1),
    qml.PauliX(0) @ qml.PauliX(1)
]

In [17]:
h = qml.Hamiltonian(coeffs=coeffs, observables=obs_list)

In [20]:
h.ops

[Tensor(expval(PauliZ(wires=[0])), Identity(wires=[1])),
 Tensor(Identity(wires=[0]), expval(PauliZ(wires=[1]))),
 Tensor(PauliX(wires=[0]), PauliX(wires=[1]))]

In [5]:
def circuit(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    
    qml.CNOT(wires=wires)
    
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])

In [24]:
qnodes = qml.map(circuit, obs_list, dev, measure='expval')
energy_expval = qml.dot([alpha, alpha, beta], qnodes)

In [7]:
step_size = 0.3
max_iterations = 200

initial_params = np.random.uniform(low=0, high=2*np.pi, size=4).tolist()

conv_tol = 1e-06

print_freq = 10

In [10]:
opt = qml.QNGOptimizer(stepsize=step_size, diag_approx=False, share_ansatz=True)

params = initial_params
prev_energy = energy_expval(params)
ngd_energies = [prev_energy]

for n in range(max_iterations):
    
    # Take step
    params = opt.step(energy_expval, params)
    
    # Compute energy
    energy = energy_expval(params)
    ngd_energies.append(energy)
    
    # Compute difference wrt previous energy (to test termination criterion)
    conv = np.abs(energy - prev_energy)

    if n % print_freq == 0:
        print('Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {'
              ':.8f} Ha'.format(n, energy, conv))

    if conv <= conv_tol:
        ngd_nsteps = n
        break

    prev_energy = energy

print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print()
print('Final circuit parameters = \n', params)
print('Number of iterations = ', ngd_nsteps)

ValueError: Objective function must be encoded as a single QNode

In [13]:
type(energy_expval)

function

In [ ]:
params = initial_params
prev_energy = energy_expval(params)
qngd_energies_block_diag = [prev_energy]

for n in range(max_iterations):
    
    grad_cost = qml.grad(energy_expval, argnum=[0])
    grad_at_point = [float(i) for i in grad_cost(params)[0]] 
    
    params = params - step_size * np.dot(np.linalg.pinv(qnodes[0].metric_tensor([params])),
                                         grad_at_point)

    energy = energy_expval(params)
    qngd_energies_block_diag.append(energy)
    
    conv = np.abs(energy - prev_energy)

    if n % print_freq == 0:
        print('Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {'
              ':.8f} Ha'.format(n, energy, conv))

    if conv <= conv_tol:
        qngd_block_diag_nsteps = n
        break

    prev_energy = energy

print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print()
print('Final circuit parameters = \n', params)
print('Number of iterations = ', qngd_block_diag_nsteps)